In [ ]:
import os
import pandas as pd
from utils import load_sol_files
from tracktour._io_util import load_tiff_frames

root_pth = '/home/ddon0001/PhD/experiments/scaled/no_merges_all'
ds_summary_path = os.path.join(root_pth, 'summary.csv')
ds_info = pd.read_csv(ds_summary_path)[['ds_name', 'det_path']]
overall_df_path = os.path.join(root_pth, 'all_edges_with_target.csv')

In [ ]:
ds_name = 'Fluo-N2DH-SIM+_01'
sol_pth = os.path.join(root_pth, ds_name)
_, all_edges, nx_sol, gt_graph, gt_to_sol, sol_to_gt = load_sol_files(sol_pth)
# res_seg = load_tiff_frames("/home/ddon0001/PhD/experiments/error_sampling_iter/FFluo-N3DH-SIM+/01_RES")

In [ ]:
divs = [node for node in gt_graph.nodes if gt_graph.out_degree(node) > 1]
len(divs)

In [ ]:
our_divs = [node for node in nx_sol.nodes if nx_sol.out_degree(node) > 1]
len(our_divs)

In [ ]:
our_divs

In [ ]:
nx_sol.nodes[290]

In [ ]:
[nx_sol.edges[edge] for edge in nx_sol.out_edges(290)]

In [ ]:
list(gt_graph.successors('11_9'))

In [ ]:
all_edges[all_edges.u == -3]['cost'].describe()

In [ ]:
all_edges[all_edges.u == -2]['cost'].describe()

In [ ]:
our_apps = [node for node in nx_sol.nodes if nx_sol.in_degree(node) == 0]
len(our_apps)

In [ ]:
[edge for edge in nx_sol.edges if nx_sol.edges[edge]['EdgeFlag.WRONG_SEMANTIC']]

In [ ]:
[nx_sol.nodes[295], nx_sol.nodes[325]]

In [ ]:
list(nx_sol.predecessors(329))

In [ ]:
gt_to_sol['33_10']

In [ ]:
gt_graph.out_edges('18_9')

In [ ]:
apps = [node for node in gt_graph.nodes if gt_graph.in_degree(node) == 0]
len(apps)

In [ ]:
skips = [edge for edge in gt_graph.edges if gt_graph.nodes[edge[0]]['t'] + 1 != gt_graph.nodes[edge[1]]['t']]
len(skips)

In [ ]:
import networkx as nx

div_merge = set()
for node in nx_sol.nodes:
    if nx_sol.in_degree(node) > 1:
        merge_edges = set(nx_sol.in_edges(node))
        div_merge.update(merge_edges)
    if nx_sol.out_degree(node) > 1:
        div_edges = set(nx_sol.out_edges(node))
        div_merge.update(div_edges)
non_div_merge = set(nx_sol.edges) - div_merge
subg = nx_sol.edge_subgraph(non_div_merge)
ccs = nx.connected_components(subg.to_undirected())
tid_dict = {}
for i, cc in enumerate(ccs, start=1):
    tid_dict.update({node: i for node in cc})
nx.set_node_attributes(nx_sol, tid_dict, 'tid')

In [ ]:
[node for node in nx_sol.nodes if nx_sol.nodes[node]['t'] == 24 and nx_sol.nodes[node]['y'] < 19]

In [ ]:
nx_sol.nodes[1337]

In [ ]:
list(nx_sol.successors(1337))

In [ ]:
import pprint
from traccuracy import run_metrics
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatcher, IOUMatcher
from traccuracy.metrics import CTCMetrics, DivisionMetrics

pp = pprint.PrettyPrinter(indent=4)

gt_data = load_ctc_data(
    '/home/ddon0001/PhD/data/cell_tracking_challenge/SUBMISSION/Fluo-N2DL-HeLa/02_GT/TRA/',
)
pred_data = load_ctc_data(
    '/home/ddon0001/PhD/experiments/error_sampling_iter/Fluo-N2DL-HeLa/02_RES/',
)
ctc_results = run_metrics(
    gt_data=gt_data, 
    pred_data=pred_data, 
    matcher=CTCMatcher(), 
    metrics=[CTCMetrics()],
)
pp.pprint(ctc_results)

In [ ]:
pred_sol = pred_data.graph
gt_sol = gt_data.graph

In [ ]:
gt_sol

In [ ]:
from traccuracy import EdgeFlag

fps = [edge for edge in pred_sol.edges if pred_sol.edges[edge][EdgeFlag.FALSE_POS]]

In [ ]:
fps

In [ ]:
list(pred_sol.successors('48_37'))

In [ ]:
pred_sol.nodes['94_24']

In [ ]:
matcher = CTCMatcher()
mapping = matcher.compute_mapping(gt_data, pred_data)
gt_to_sol = {}
sol_to_gt = {}
for gt, pred in mapping.mapping:
    gt_to_sol[gt] = pred
    sol_to_gt[pred] = gt

In [ ]:
sol_to_gt['48_37']

In [ ]:
list(gt_sol.successors('24_37'))

In [ ]:
from traccuracy.loaders import load_ctc_data
pth = '/home/ddon0001/PhD/experiments/resolve_sampling/Fluo-N3DH-CE/02_RES'
all_edges_pth = '/home/ddon0001/PhD/experiments/resolve_sampling/Fluo-N3DH-CE/02_all_edges.csv'

sol_g = load_ctc_data(pth).graph

In [ ]:
merges = [node for node in sol_g.nodes if sol_g.in_degree(node) > 1]
len(merges)

In [ ]:
triple_children = [node for node in sol_g.nodes if sol_g.out_degree(node) > 2]
len(triple_children)

In [ ]:
import pandas as pd
all_edges = pd.read_csv(all_edges_pth)
correct = all_edges[all_edges.oracle_is_correct == 1]
assert correct.flow.sum() == len(correct)
incorrect = all_edges[all_edges.oracle_is_correct == 0]
assert incorrect.flow.sum() == 0
fractional = all_edges[all_edges.flow % 1 != 0]
assert len(fractional) == 0

In [1]:
import pprint
from traccuracy import run_metrics
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatcher, IOUMatcher
from traccuracy.metrics import CTCMetrics, DivisionMetrics

pp = pprint.PrettyPrinter(indent=4)

gt_data = load_ctc_data(
    '/home/ddon0001/PhD/data/cell_tracking_challenge/SUBMISSION/PhC-C2DL-PSC/01_GT/TRA/',
)
pred_data = load_ctc_data(
    '/home/ddon0001/PhD/experiments/resolve_sampling/PhC-C2DL-PSC/01_RES',
)
ctc_results = run_metrics(
    gt_data=gt_data, 
    pred_data=pred_data, 
    matcher=CTCMatcher(), 
    metrics=[CTCMetrics()],
)
pp.pprint(ctc_results)

Loading TIFFs: 100%|██████████| 299/299 [00:00<00:00, 589.39it/s]
2 non-connected masks at t=286.
1 non-connected masks at t=287.
Evaluating FN edges: 100%|██████████| 71201/71201 [00:00<00:00, 370054.77it/s]


[   {   'matcher': {'name': 'CTCMatcher'},
        'metric': {   'e_weights': {'fn': 1.5, 'fp': 1, 'ws': 1},
                      'name': 'CTCMetrics',
                      'v_weights': {'fn': 10, 'fp': 1, 'ns': 5}},
        'results': {   'AOGM': 5635.0,
                       'DET': 0.9978572328893744,
                       'TRA': 0.9931350100477382,
                       'fn_edges': 1460,
                       'fn_nodes': 153,
                       'fp_edges': 1388,
                       'fp_nodes': 0,
                       'ns_nodes': 0,
                       'ws_edges': 527},
        'version': '0.1.1.dev28+gb561814.d20240617'}]


In [ ]:
# PhC-C2DL-PSC/01
og = {
    "fn_nodes": 336,
    "fp_edges": 1161,
    "fn_edges": 1421,
    "ws_edges": 513}
resolve = {
    'fn_nodes': 153,
    'fp_edges': 1388,
    'fn_edges': 1460,
    'ws_edges': 527}
no_resolve = {
    'fn_nodes': 153,
    'fp_edges': 0,
    'fn_edges': 666,
    'ws_edges': 194}



In [2]:
import pprint
from traccuracy import run_metrics
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatcher, IOUMatcher
from traccuracy.metrics import CTCMetrics, DivisionMetrics
gt_data = load_ctc_data(
    '/home/ddon0001/PhD/data/cell_tracking_challenge/SUBMISSION/Fluo-N2DH-SIM+/02_GT/TRA/',
)
pred_data = load_ctc_data(
    '/home/ddon0001/PhD/experiments/error_sampling_ws/Fluo-N2DH-SIM+/02_RES',
)
ctc_results = run_metrics(
    gt_data=gt_data, 
    pred_data=pred_data,
    matcher=CTCMatcher(), 
    metrics=[CTCMetrics()],
)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(ctc_results[0]['results'])

Loading TIFFs: 100%|██████████| 149/149 [00:00<00:00, 166.11it/s]
1 non-connected masks at t=14.
1 non-connected masks at t=16.
1 non-connected masks at t=17.
1 non-connected masks at t=20.
2 non-connected masks at t=23.
2 non-connected masks at t=24.
3 non-connected masks at t=26.
1 non-connected masks at t=27.
1 non-connected masks at t=28.
1 non-connected masks at t=29.
3 non-connected masks at t=30.
2 non-connected masks at t=31.
3 non-connected masks at t=32.
2 non-connected masks at t=33.
1 non-connected masks at t=43.
2 non-connected masks at t=45.
2 non-connected masks at t=46.
2 non-connected masks at t=49.
1 non-connected masks at t=50.
1 non-connected masks at t=52.
2 non-connected masks at t=56.
1 non-connected masks at t=57.
2 non-connected masks at t=61.
1 non-connected masks at t=62.
1 non-connected masks at t=65.
1 non-connected masks at t=66.
1 non-connected masks at t=68.
2 non-connected masks at t=69.
5 non-connected masks at t=71.
2 non-connected masks at t=72.
1 no

{   'AOGM': 36.5,
    'DET': 1.0,
    'TRA': 0.9990578456931932,
    'fn_edges': 23,
    'fn_nodes': 0,
    'fp_edges': 0,
    'fp_nodes': 0,
    'ns_nodes': 0,
    'ws_edges': 2}


In [ ]:
distance_final = {   'AOGM': 34.5,
    'DET': 1.0,
    'TRA': 0.999109470586717,
    'fn_edges': 23,
    'fn_nodes': 0,
    'fp_edges': 0,
    'fp_nodes': 0,
    'ns_nodes': 0,
    'ws_edges': 0}

sens_final = {   'AOGM': 34.5,
    'DET': 1.0,
    'TRA': 0.999109470586717,
    'fn_edges': 23,
    'fn_nodes': 0,
    'fp_edges': 0,
    'fp_nodes': 0,
    'ns_nodes': 0,
    'ws_edges': 0}

In [11]:
import pprint
from traccuracy import run_metrics
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatcher, IOUMatcher
from traccuracy.metrics import CTCMetrics, DivisionMetrics
pp = pprint.PrettyPrinter(indent=4)

new_gt_data = load_ctc_data(
    '/home/ddon0001/PhD/data/cell_tracking_challenge/SUBMISSION/Fluo-N2DH-GOWT1/02_GT/TRA/',
)
new_pred_data = load_ctc_data(
    '/home/ddon0001/PhD/experiments/error_sampling_ws_fa_fe/Fluo-N2DH-GOWT1/02_RES',
)
ctc_results = run_metrics(
    gt_data=new_gt_data, 
    pred_data=new_pred_data, 
    matcher=CTCMatcher(), 
    metrics=[CTCMetrics()],
)
pp.pprint(ctc_results[0]['results'])

Loading TIFFs:   0%|          | 0/91 [00:00<?, ?it/s]

Evaluating FN edges: 100%|██████████| 2472/2472 [00:00<00:00, 387876.23it/s]

{   'AOGM': 149.0,
    'DET': 0.9944378228049265,
    'TRA': 0.9948403629060184,
    'fn_edges': 6,
    'fn_nodes': 14,
    'fp_edges': 0,
    'fp_nodes': 0,
    'ns_nodes': 0,
    'ws_edges': 0}


In [12]:
from traccuracy import NodeFlag
fn_nodes = [node for node in new_gt_data.graph.nodes if new_gt_data.graph.nodes[node][NodeFlag.FALSE_NEG]]
fn_nodes

['6_46',
 '6_47',
 '6_48',
 '39_0',
 '39_1',
 '39_2',
 '39_3',
 '40_0',
 '40_1',
 '50_20',
 '41_2',
 '44_6',
 '48_17',
 '46_10']

In [15]:
from traccuracy import EdgeFlag
fn_edges = [edge for edge in new_gt_data.graph.edges if new_gt_data.graph.edges[edge][EdgeFlag.FALSE_NEG]]
fn_edges

[('6_46', '6_47'),
 ('6_47', '6_48'),
 ('39_0', '39_1'),
 ('39_1', '39_2'),
 ('39_2', '39_3'),
 ('40_0', '40_1')]

In [19]:
list(new_gt_data.graph.successors('40_1'))

[]

In [14]:
nids = ['6_', '39_', '40_', '41_', '44_', '50_', '46_', '48_']

for nid in nids:
    track_nodes = [node for node in new_gt_data.graph.nodes if node.startswith(nid)]
    unfound = [node for node in track_nodes if node in fn_nodes]
    # if len(track_nodes) != len(unfound):
    print(f"{nid}: {len(track_nodes)}, {len(unfound)}")
    # print(unfound)

6_: 3, 3
39_: 4, 4
40_: 2, 2
41_: 1, 1
44_: 1, 1
50_: 1, 1
46_: 1, 1
48_: 1, 1


In [6]:
matcher = CTCMatcher().compute_mapping(new_gt_data, new_pred_data)
sol_to_gt = {}
gt_to_sol = {}
for gt, pred in matcher.mapping:
    sol_to_gt[pred] = gt
    gt_to_sol[gt] = pred

Matching frames: 100%|██████████| 92/92 [00:00<00:00, 108.33it/s]


In [9]:
gt_to_sol['21_77']


'36_77'

In [10]:
list(new_pred_data.graph.predecessors('36_77'))

[]

In [5]:
import pprint
from traccuracy import run_metrics
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatcher, IOUMatcher
from traccuracy.metrics import CTCMetrics, DivisionMetrics
pp = pprint.PrettyPrinter(indent=4)

gowt_1_gt = load_ctc_data(
    '/home/ddon0001/PhD/data/cell_tracking_challenge/SUBMISSION/Fluo-N2DH-GOWT1/01_GT/TRA/',
)
gowt_1_pred = load_ctc_data(
    '/home/ddon0001/PhD/experiments/error_sampling_ws_fa_fe/Fluo-N2DH-GOWT1/01_RES',
)
ctc_results = run_metrics(
    gt_data=gowt_1_gt, 
    pred_data=gowt_1_pred, 
    matcher=CTCMatcher(), 
    metrics=[CTCMetrics()],
)
pp.pprint(ctc_results[0]['results'])

Evaluating FN edges: 100%|██████████| 2035/2035 [00:00<00:00, 832641.56it/s]

{   'AOGM': 0.0,
    'DET': 1.0,
    'TRA': 1.0,
    'fn_edges': 0,
    'fn_nodes': 0,
    'fp_edges': 0,
    'fp_nodes': 0,
    'ns_nodes': 0,
    'ws_edges': 0}


In [6]:
[node for node in gowt_1_gt.graph.nodes if gowt_1_gt.graph.out_degree(node) > 1]

['10_46', '16_16']

In [7]:
[node for node in gowt_1_pred.graph.nodes if gowt_1_pred.graph.out_degree(node) > 1]

['9_16', '22_46']